Autoreload notebook

In [65]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


>> ### Import required Libraries

In [1]:
# Import packages

import pandas as pd
import numpy as np
import random
import os
import sys

#notebook reproducible 
np.random.seed(42)
random.seed(42)

# Directory listing
from glob import glob

# Visualization
import matplotlib.pylab as plt
import seaborn as sns

# Audio Processing
import librosa
import librosa.display
import IPython.display as ipd
from pydub import AudioSegment

# Color Palettes

from itertools import cycle

sns.set_theme(style="white", palette=None)
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

>> #### Import modules

In [7]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from data_preprocessing import AudioUtil
from data_loader import DataLoader
from model import STTModel

Initialize scripts

In [8]:
DATA_DIR = '../data/'

In [66]:
loader = DataLoader(data_dir='../data/data/', sample_rate=16000, max_duration=10.0)

In [61]:
prep = AudioUtil()

In [118]:
model = STTModel(loader)

Load MetaData

In [32]:
train_data = loader.read_csv('../data/metadata.csv')

In [33]:
train_data.head()

,key,text,duration
0,../data/AMHARIC/data/train/wav/tr_10000_tr0970...,የተለያዩ የ ትግራይ አውራጃ ተወላጆች ገንዘባቸው ን አዋጥ ተው የ ልማት ...,9.088
1,../data/AMHARIC/data/train/wav/tr_10001_tr0970...,የ ጠመንጃ ተኩስ ተከፈተ ና አራት የኤርትራ ወታደሮች ተገደሉ,5.632
2,../data/AMHARIC/data/train/wav/tr_10002_tr0970...,ላነሷቸው ጥያቄዎች የ ሰጡት ን መልስ አቅርበ ነዋል,6.144
3,../data/AMHARIC/data/train/wav/tr_10003_tr0970...,እ ብዱ አስፋልቱ ላይ የ ኰለኰ ለ ው ድንጋይ መኪና አላ ሳልፍ አለ,5.760
4,../data/AMHARIC/data/train/wav/tr_10004_tr0970...,ጠጁ ን ኰ መኰ መ ኰ መኰ መ ና ሚስቱ ን ሲ ያሰቃ ያት አደረ,5.376


In [34]:
audio_paths = train_data['key']

In [35]:
audio_paths = audio_paths.to_list()

In [36]:
AUDIO_LEN = train_data['key'].shape[0]

In [37]:
# Load json data
train_json = "../data/train.json"

In [38]:
data_est = prep.mean_est(AUDIO_LEN, audio_paths)

In [119]:
# Training
# from src.train import train
# from src.models import model_2
# from src.char_map import char_map, index_map



EPOCHS = 40
MODEL_NAME = "model_2"
MINI_BATCH_SIZE = 250
# input_dim=13,
#                 filters=200,
#                 kernel_size=11, 
#                 conv_stride=2,
#                 conv_border_mode='valid',
#                 units=250,
#                 activation='relu',
#                 dropout_rate=1,
#                 number_of_layers=5,
#                 output_dim=len(char_map)+1
char_map, _ = loader.map_index()
model_tr = model.model(input_dim=13,
                filters=200,
                kernel_size=11, 
                conv_stride=2,
                conv_border_mode='valid',
                units=250,
                activation='relu',
                dropout_rate=0.2,
                number_of_layers=5,
                output_dim=len(char_map)+1)

Model: "model_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 the_input (InputLayer)      [(None, None, 13)]        0         
                                                                 
 layer_1_conv (Conv1D)       (None, None, 200)         28800     
                                                                 
 conv_batch_norm (BatchNorma  (None, None, 200)        800       
 lization)                                                       
                                                                 
 rnn_1 (GRU)                 (None, None, 250)         339000    
                                                                 
 bt_rnn_1 (BatchNormalizatio  (None, None, 250)        1000      
 n)                                                              
                                                                 
 rnn_2 (GRU)                 (None, None, 250)         376

In [120]:
model.train(audio_paths, input_to_softmax=model_tr, model_name=MODEL_NAME, epochs=EPOCHS, minibatch_size=MINI_BATCH_SIZE)

the shape of the features is:  100
First one sample is:  [[16.48025134 -7.29267008  2.70881331 ...  3.21228454  2.26029197
   1.14607319]
 [15.51330106  3.68151116 21.16174352 ...  4.33809444  5.41812124
   1.70194754]
 [15.50275689  3.30036992 21.75593785 ...  1.42770988  6.47290421
   2.80015671]
 ...
 [15.45351174  3.3376891  19.44328789 ...  2.04839206  2.38379533
   4.20881466]
 [15.48023503  3.86178501 18.38389025 ...  4.07211     4.06424513
   7.41911657]
 [15.16704021  4.47106555 18.018563   ...  4.67307592  4.59965818
   9.04049016]]
Epoch 1/40
